In [1]:
import urllib.request, json 
import requests
import pandas as pd

from bs4 import BeautifulSoup

pd.set_option('display.max_rows',None, 'display.max_columns',None)

# SCRAP DATA DCT DPRD KAB

In [6]:
def save_profile(profile):
    #save table dengan tipe td dan tr
    td_element = profile.find_all("td")

    for i in range(1, len(td_element), 2):  
        key = td_element[i-1].text.strip().replace(":", "")
        value = td_element[i].text.strip()

        #menyimpan informasi caleg kedalam dict
        if key in data_caleg:
            data_caleg[key].append(value)
        else:
            data_caleg[key] = [value]
    
    return data_caleg

def save_address(address):
    cek = address.find_all("li", {"class": "list-group-item"})

    for li_element in cek:
        strong_element = li_element.find("strong")
        if strong_element:
            key = strong_element.text.strip().replace(":", "")
            value = li_element.contents[-1].strip()
            
            # Menyimpan informasi ke dalam dictionary
            if key in data_caleg:
                data_caleg[key].append(value)
            else:
                data_caleg[key] = [value]
    
    return data_caleg

def save_pekerjaan(pekerjaan):
    
    for h3_element in pekerjaan:
        
        key = h3_element.text.strip()
        p_element = h3_element.find_next_sibling("p")
        if p_element:
            value = p_element.text.strip()

            if key in data_caleg:
                data_caleg[key].append(value)
            else:
                data_caleg[key] = [value]
    
    return data_caleg

def save_pendidikan(pendidikan, data_dict):
    #save tipe data table dengan thead dan tbody
    table_pendidikan = pendidikan.find_all("table")
    
    for table_element in table_pendidikan:
        headers = table_element.find_all('th')
        key = [th.text.strip() for th in headers]

        td_elements = table_element.select('tbody tr')
        rows = [[td.text.strip() for td in row.select('td')] for row in td_elements]
        update_rows = [row + ['-'] * (5 - len(row)) for row in rows]

        for x, values in zip(key,list(zip(*update_rows))):
            
            if x in data_dict:
                data_dict[x].append(values)
            else:
                data_dict[x] = [values]
    
    return data_dict

def extract_party_name(item):
    soup = BeautifulSoup(item, 'html.parser')
    party_name = soup.text.strip().replace("Dapil","")
    return party_name

def save_data(kursus_dict, pekerjaan_dict, organisasi_dict, data_caleg, caleg_tidak_bersedia, data_ke):
    tidak_bersedia_df = pd.DataFrame.from_dict(caleg_tidak_bersedia, orient='index')
    tidak_bersedia_df.T.to_excel(f"../DCT/result_scrap/dprd_kab/tidak_bersedia{data_ke}.xlsx")

    kursus_df = pd.DataFrame.from_dict(kursus_dict, orient='index')
    pekerjaan_df = pd.DataFrame.from_dict(pekerjaan_dict, orient='index')
    organisasi_df = pd.DataFrame.from_dict(organisasi_dict, orient='index')
    caleg_df = pd.DataFrame.from_dict(data_caleg, orient='index')

    df_list = [caleg_df.T, pekerjaan_df.T, organisasi_df.T, kursus_df.T]
    data_dprd_kab = pd.concat(df_list, axis=1)
    data_dprd_kab.to_excel(f"../DCT/result_scrap/dprd_kab/result_dprd_kab{data_ke}.xlsx", index=False)

In [ ]:
respon = requests.get("https://infopemilu.kpu.go.id/Pemilu/Dct_dprd/GetDapilOptions_dprdkabko")
kode_dapil = []

for kode in respon.json()['data']:
    kode_dapil.append(kode['kode_dapil'])

In [ ]:
link_dprd_kab = "https://infopemilu.kpu.go.id/Pemilu/Dct_dprd/Dct_dprdkabko?kode_dapil="
profile = 'https://infopemilu.kpu.go.id/Pemilu/Dct_dprd/profile'

data_caleg = {'kode_dapil':[]}
caleg_tidak_bersedia = {'kode_dapil':[]}
pekerjaan_dict = {}
kursus_dict = {}
organisasi_dict = {}
selesai = 240
insert = 0
save_caleg = 0
kode_dapil_selesai = []

max_attemp = 3
attemp = 0
while attemp < max_attemp:
  try:
      for i,x in enumerate(kode_dapil[240:]):

          print("kode dapil", x)
          coba = requests.get(link_dprd_kab+x)
          for i, j in enumerate(coba.json()['data']):

              soup = BeautifulSoup(j[8], 'html.parser')
              tag_a = soup.find_all('a', {'class':'btn btn-danger'})

              nama = 'Nama_Lengkap'
              jk = 'Jenis Kelamin'
              nama_partai = 'Nama Partai'
              nama_dapil = 'Dapil'
              nama_pro = 'Kabupaten/Kota'

              partai = extract_party_name(j[0])
              dapil = extract_party_name(j[1])
              provinsi = j[6]

              if tag_a:
                  #jika profile caleg tidak bersedia
                  print("proses caleg tidak bersedia")
                  caleg_tidak_bersedia['kode_dapil'].append(x)
                  if nama in caleg_tidak_bersedia or jk in caleg_tidak_bersedia or nama_partai in caleg_tidak_bersedia or nama_dapil in caleg_tidak_bersedia or nama_pro in caleg_tidak_bersedia:
                      caleg_tidak_bersedia[nama].append(j[4])
                      caleg_tidak_bersedia[jk].append(j[5])
                      caleg_tidak_bersedia[nama_partai].append(partai)
                      caleg_tidak_bersedia[nama_dapil].append(dapil)
                      caleg_tidak_bersedia[nama_pro].append(provinsi)
                  else:
                      caleg_tidak_bersedia[nama] = [j[4]]
                      caleg_tidak_bersedia[jk] = [j[5]]
                      caleg_tidak_bersedia[nama_partai] = [partai]
                      caleg_tidak_bersedia[nama_dapil] = [dapil]
                      caleg_tidak_bersedia[nama_pro] = [provinsi]
                  print("caleg tidak bersedia selesai di proses !!!!!")
              else:
                  #jika profile caleg bersedia
                  data_caleg['kode_dapil'].append(x)
                  if nama_partai in data_caleg or nama_dapil in data_caleg:
                      data_caleg[nama_partai].append(partai)
                      data_caleg[nama_dapil].append(dapil)
                  else:
                      data_caleg[nama_partai] = [partai]
                      data_caleg[nama_dapil] = [dapil]

                  id_dpr = soup.find('input', {'id': 'id_calon_dpr'})['value']
                  params = {'id_calon_dpr':id_dpr, 'pilihan_publikasi' : 'datadiri,alamat,pekerjaan,hukum,pendidikan,riwayatpekerjaan,kursus,organisasi', 'status_publikasi' : 'Bersedia Sebagian'}
                  get_data = requests.post(profile, data=params)
                  soup_profile = BeautifulSoup(get_data.content, 'html.parser')

                  card_profile = soup_profile.find_all("div", {"class": "card"})[4]
                  container = card_profile.find("div", {"class": "container"})
                  table_profile = container.find("table", {"class":"table table-striped"})
                  save_profile(table_profile)

                  alamat = container.find_next("div", {"class": "container"})
                  save_address(alamat)

                  pekerjaan = alamat.find_next_sibling("div", {"class": "table-responsive"})
                  save_pekerjaan(pekerjaan)

                  status_hukum = pekerjaan.find_next_sibling("div")
                  save_pekerjaan(status_hukum)

                  riwayat_pendidikan = status_hukum.find_next_sibling("div")
                  save_pendidikan(riwayat_pendidikan, data_caleg)

                  riwayat_pekerjaan = riwayat_pendidikan.find_next_sibling("div")
                  save_pendidikan(riwayat_pekerjaan, pekerjaan_dict)

                  riwayat_kursus = riwayat_pekerjaan.find_next_sibling("div")
                  save_pendidikan(riwayat_kursus, kursus_dict)

                  riwayat_organisasi = riwayat_kursus.find_next_sibling("div")
                  save_pendidikan(riwayat_organisasi, organisasi_dict)

                  insert += 1
                  print("insert data ke", insert)

          selesai+=1
          print(x, "Selesai")
          print(selesai)
          kode_dapil_selesai.append(x)
          data_caleg['kode_dapil']

          save_caleg += 1
          if selesai % 5 == 0:
              save_data(kursus_dict, pekerjaan_dict, organisasi_dict, data_caleg, caleg_tidak_bersedia,save_caleg)
              print("save caleg to excel")

  except KeyboardInterrupt as e:
    save_data(kursus_dict, pekerjaan_dict, organisasi_dict, data_caleg, caleg_tidak_bersedia,99)
    print("Kode diberhentikan")
  except (IndexError, ConnectionAbortedError) as t:
    print("Mencoba Terhubung Kembali ...")
    attemp += 1
    time.sleep(10)
    
else:
  print("script stop karna sudah melebihi percobaan")

In [ ]:
save_data(kursus_dict, pekerjaan_dict, organisasi_dict, data_caleg, caleg_tidak_bersedia,11)